In [1]:
# %pip install k-llms

Retab's `k-LLMs` library is a library dedicated to the `n_consensus` parameter in retab's SDK.

**CONCEPT**—Large language models are **non-deterministic**—the same prompt can return different answers each time. **Consensus reduces this variability** by sending the prompt through K independent model calls and merging their JSON outputs into one result.

**CONTEXT**—Our approach follows similar principles to **[Palantir’s “K-LLMs” methodology](https://www.linkedin.com/posts/palantir-technologies_never-use-1-llm-when-you-can-use-k-llms-activity-7109622372021780480-eMPG/)**, where multiple models evaluate the same prompt and their outputs are synthesized for increased accuracy, confidence, and reduced hallucinations.

**SOLUTION**—We provide a quick, type‑safe wrapper around **OpenAI Chat Completions and Responses** endpoints with automatic consensus reconciliation.

**For more information on k-LLMs Consensus, check our [documentation](https://docs.retab.com/core-concepts/k-LLMs_Consensus).**

In [ ]:
# Parse the document with Retab
from dotenv import load_dotenv
from retab import Retab

load_dotenv() # You need to create a .env file containing your OPENAI_API_KEY=sk-***

reclient = Retab()

parsed_invoice = reclient.documents.parse(
    document="../assets/docs/invoice.jpeg",
    model="gemini-2.5-flash",
    table_parsing_format="markdown",  # Better for RAG
    image_resolution_dpi=128          # Higher quality for technical docs
)

In [ ]:
# Use the k-LLMs library to gain accuracy

from k_llms import KLLMs
from pydantic import BaseModel

client = KLLMs()

class Invoice(BaseModel):
    status: str
    total: str

completion = client.chat.completions.parse(
    model="gpt-5",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": parsed_invoice.pages[0]},
    ],
    response_format=Invoice,
    n=5
)

print("Consensus output:", completion.choices[0].message.parsed)
print("Likelihoods:", completion.likelihoods)

for i in range(len(completion.choices)):
    print(f"Choice {i}: {completion.choices[i].message.parsed}")

Consensus helps us spot inconsistencies on specific fields.

These inconsistencies can be tackled by adding `Reasoning` to the Field, as shown in our **03.reasoning-prompt.ipynb** Notebook for example.

**For more information on `Reasoning`, check our [documentation](https://docs.retab.com/core-concepts/Reasoning).**